<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [68]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv(r"C:\\Users\\Tamanna Kawatra\\OneDrive\\Desktop\\netflix_titles.csv")
new_df = df[['title','director','cast', 'listed_in','description']]
new_df.head()

,title,director,cast,listed_in,description
0,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,Jandino: Whatever it Takes,NaN,Jandino Asporaat,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",Kids' TV,"With the help of three human allies, the Autob..."
3,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",Kids' TV,When a prison ship crash unleashes hundreds of...
4,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",Comedies,When nerdy high schooler Dani finally attracts...


In [69]:
# initializing the new column
new_df['Key_words'] = ""
new_df.head()

for index, row in new_df.iterrows():
    plot = row['description']
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column for the corresponding movie
    row['Key_words'] = list(key_words_dict_scores.keys())



# dropping the Plot column
new_df.drop(columns = ['description'], inplace = True)
new_df.head()
new_df.shape

C:\Users\Tamanna Kawatra\Anaconda3\envs\buan6341_2020\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Tamanna Kawatra\Anaconda3\envs\buan6341_2020\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


(6234, 5)

In [70]:
 #REMOVE NaN VALUES AND EMPTY STRINGS:
new_df.dropna(inplace=True)

blanks = []  # start with an empty list

col=['title','director','cast','listed_in','description']
for i,col in new_df.iterrows():  # iterate over the DataFrame
    if type(col)==str:            # avoid NaN values
        if col.isspace():         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list

new_df.drop(blanks, inplace=True)


# discarding the commas between the actors' full names and getting only the first three names
new_df['cast'] = new_df['cast'].map(lambda x: x.split(',')[:3])

# putting the genres in a list of words
new_df['listed_in'] = new_df['listed_in'].map(lambda x: x.lower().split(','))

new_df['director'] = new_df['director'].map(lambda x: x.split(' '))

# merging together first and last name for each actor and director, so it's considered as one word 
# and there is no mix up between people sharing a first name
for index, row in new_df.iterrows():
    row['cast'] = [x.lower().replace(' ','') for x in row['cast']]
    row['director'] = ''.join(row['director']).lower()


C:\Users\Tamanna Kawatra\Anaconda3\envs\buan6341_2020\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Tamanna Kawatra\Anaconda3\envs\buan6341_2020\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Tamanna Kawatra\Anaconda3\envs\buan6341_2020\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/st

In [71]:
new_df.set_index('title', inplace = True)
new_df.head()

,director,cast,listed_in,Key_words
title,,,,
Norm of the North: King Sized Adventure,"richardfinn,timmaltby","[alanmarriott, andrewtoth, briandobson]","[children & family movies, comedies]","[awesome, wedding, polar, bear, king, must, ta..."
#realityhigh,fernandolebrija,"[nestacooper, katewalsh, johnmichaelhiggins]",[comedies],"[cross, hairs, longtime, crush, nerdy, high, s..."
Automata,gabeibáñez,"[antoniobanderas, dylanmcdermott, melaniegriff...","[international movies, sci-fi & fantasy, thr...","[discovers, global, conspiracy, robot, killed,..."
Fabrizio Copano: Solo pienso en mi,"rodrigotoro,franciscoschultz",[fabriziocopano],[stand-up comedy],"[reflecting, set, sperm, banks, stand, fabrizi..."
Good People,henrikrubengenz,"[jamesfranco, katehudson, tomwilkinson]","[action & adventure, thrillers]","[stash, recently, murdered, apartment, struggl..."


In [72]:
new_df['bag_of_words'] = ''
columns = new_df.columns
for index, row in new_df.iterrows():
    words = ''
    for col in columns:
        if col != 'director':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    row['bag_of_words'] = words
    
new_df.head()


C:\Users\Tamanna Kawatra\Anaconda3\envs\buan6341_2020\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,director,cast,listed_in,Key_words,bag_of_words
title,,,,,
Norm of the North: King Sized Adventure,"richardfinn,timmaltby","[alanmarriott, andrewtoth, briandobson]","[children & family movies, comedies]","[awesome, wedding, polar, bear, king, must, ta...","richardfinn,timmaltby alanmarriott andrewtoth ..."
#realityhigh,fernandolebrija,"[nestacooper, katewalsh, johnmichaelhiggins]",[comedies],"[cross, hairs, longtime, crush, nerdy, high, s...",fernandolebrija nestacooper katewalsh johnmich...
Automata,gabeibáñez,"[antoniobanderas, dylanmcdermott, melaniegriff...","[international movies, sci-fi & fantasy, thr...","[discovers, global, conspiracy, robot, killed,...",gabeibáñez antoniobanderas dylanmcdermott mela...
Fabrizio Copano: Solo pienso en mi,"rodrigotoro,franciscoschultz",[fabriziocopano],[stand-up comedy],"[reflecting, set, sperm, banks, stand, fabrizi...","rodrigotoro,franciscoschultz fabriziocopano st..."
Good People,henrikrubengenz,"[jamesfranco, katehudson, tomwilkinson]","[action & adventure, thrillers]","[stash, recently, murdered, apartment, struggl...",henrikrubengenz jamesfranco katehudson tomwilk...


In [73]:
new_df.drop(columns = [col for col in new_df.columns if col!= 'bag_of_words'], inplace = True)
new_df.head()

C:\Users\Tamanna Kawatra\Anaconda3\envs\buan6341_2020\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,bag_of_words
title,
Norm of the North: King Sized Adventure,"richardfinn,timmaltby alanmarriott andrewtoth ..."
#realityhigh,fernandolebrija nestacooper katewalsh johnmich...
Automata,gabeibáñez antoniobanderas dylanmcdermott mela...
Fabrizio Copano: Solo pienso en mi,"rodrigotoro,franciscoschultz fabriziocopano st..."
Good People,henrikrubengenz jamesfranco katehudson tomwilk...


In [74]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(new_df['bag_of_words'])
print(count_matrix)
print(count_matrix.shape)

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(new_df.index)
indices[:5]

  (0, 17380)	1
  (0, 20897)	1
  (0, 700)	1
  (0, 1233)	1
  (0, 2979)	1
  (0, 3861)	1
  (0, 7116)	1
  (0, 14166)	1
  (0, 4345)	1
  (0, 1990)	1
  (0, 22247)	1
  (0, 16020)	1
  (0, 2258)	1
  (0, 11520)	1
  (0, 14244)	1
  (0, 20397)	1
  (0, 2051)	1
  (0, 6894)	1
  (0, 1569)	1
  (0, 7389)	1
  (0, 19880)	1
  (0, 1700)	1
  (0, 8278)	1
  (0, 15954)	1
  (1, 4345)	1
  :	:
  (3907, 22271)	1
  (3908, 4345)	1
  (3908, 9676)	1
  (3908, 17727)	1
  (3908, 21523)	1
  (3908, 20973)	1
  (3908, 22677)	1
  (3908, 3026)	1
  (3908, 12147)	1
  (3908, 21389)	3
  (3908, 18995)	2
  (3908, 5570)	1
  (3908, 21377)	1
  (3908, 11878)	1
  (3908, 22816)	1
  (3908, 5575)	1
  (3908, 13447)	1
  (3908, 17127)	1
  (3908, 858)	1
  (3908, 15071)	1
  (3908, 18673)	1
  (3908, 13376)	1
  (3908, 12120)	1
  (3908, 14556)	1
  (3908, 4535)	1
(3909, 22934)


0    Norm of the North: King Sized Adventure
1                               #realityhigh
2                                   Automata
3         Fabrizio Copano: Solo pienso en mi
4                                Good People
Name: title, dtype: object

In [75]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.04454354, 0.04166667, ..., 0.        , 0.        ,
        0.03450328],
       [0.04454354, 1.        , 0.        , ..., 0.        , 0.03253   ,
        0.03688556],
       [0.04166667, 0.        , 1.        , ..., 0.03311331, 0.03042903,
        0.03450328],
       ...,
       [0.        , 0.        , 0.03311331, ..., 1.        , 0.31437304,
        0.38388595],
       [0.        , 0.03253   , 0.03042903, ..., 0.31437304, 1.        ,
        0.32756921],
       [0.03450328, 0.03688556, 0.03450328, ..., 0.38388595, 0.32756921,
        1.        ]])

In [76]:
# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations(Title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == Title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(new_df.index)[i])
        
    return recommended_movies

In [77]:
recommendations('The Taking of Pelham 123')

['Kidnapping Mr. Heineken',
 'Jarhead: Law of Return',
 'Hunt to Kill',
 'Small Soldiers',
 'From Paris with Love',
 'Thong Dee Fun Khao',
 'Black Site Delta',
 'Set Up',
 'Kickboxer: Vengeance',
 'The Last Boy Scout']

In [83]:
recommendations('Taare Zameen Par')

['Loving Annabelle',
 'Rang De Basanti',
 'Handsome Devil',
 'Siddhant',
 'Kucch To Hai',
 'Dangal',
 'Noblemen',
 'Raja Hindustani',
 '3 Idiots',
 'Mucize']

In [79]:
recommendations('Scott Pilgrim vs. the World')

['Rumble in the Bronx',
 'Mujhse Shaadi Karogi',
 'Rowdy Rathore',
 'The Man Who Feels No Pain',
 'Asu Mare 2',
 'Love on Delivery',
 'Superlopez',
 'Maza Pati Karodpati',
 'Ip Man 3',
 'The Five Venoms']

In [80]:
recommendations('Sanju')

['Azhar',
 'Soorma',
 'Bollywood Calling',
 'Radiopetti',
 'Raajneeti',
 'The Reconquest',
 'The spiral',
 'Paheli',
 'Theeram',
 'Ghulam-E-Musthafa']

In [81]:
recommendations('You')

['Jack Taylor',
 'Father Brown',
 'Sadqay Tumhare',
 'The Fear',
 'Camelia la Texana',
 'Innocent',
 'Reply 1997',
 'Bolívar',
 'Cheese in the Trap',
 'Jamtara - Sabka Number Ayega']